In [1]:
from model.module.path_header import *

import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split

import os
from datetime import datetime

In [2]:
TRAIN_NAME = '0906-21-train-v2.csv'
TEST1_NAME = '0906-21-test1-v2.csv'
TEST2_NAME = '0906-21-test2-v2.csv'
LABEL_NAME = 'train_label.csv'

TRAIN_PATH = os.path.join(PREPROCESS_DIR, TRAIN_NAME)
TEST1_PATH = os.path.join(PREPROCESS_DIR, TEST1_NAME)
TEST2_PATH = os.path.join(PREPROCESS_DIR, TEST2_NAME)
LABEL_PATH = os.path.join(PREPROCESS_DIR, LABEL_NAME)

train = pd.read_csv(TRAIN_PATH)
test1 = pd.read_csv(TEST1_PATH)
test2 = pd.read_csv(TEST2_PATH)
label = pd.read_csv(LABEL_PATH)

In [3]:
tra = train.copy()
te1 = test1.copy()
te2 = test2.copy()
lab = label.copy()

In [4]:
tra = tra.drop('day', axis=1).groupby('acc_id').mean()
te1 = te1.drop('day', axis=1).groupby('acc_id').mean()
te2 = te2.drop('day', axis=1).groupby('acc_id').mean()

In [5]:
lab['isSurvival'] = lab['survival_time'].transform(lambda x: 1 if x==64 else 0)
lab = lab['isSurvival']

In [6]:
# mm = MinMaxScaler()
# tra_scaled = mm.fit_transform(tra)
# te1_scaled = mm.transform(te1)
# te2_scaled = mm.transform(te2)

rb = RobustScaler()
tra_scaled = rb.fit_transform(tra)
te1_scaled = rb.transform(te1)
te2_scaled = rb.transform(te2)

In [7]:
tra_X, val_X, tra_y, val_y = train_test_split(tra_scaled, lab, train_size=0.8)

In [22]:
# gridsearch
param_neighbors=30
param_algorithm= ['auto', 'ball_tree', 'kd_tree', 'brute']

print(':::::::: GridSearch')
for algo in param_algorithm:
    for neighbors in range(3, param_neighbors, 2):
        neigh = KNeighborsClassifier(algorithm=algo, n_neighbors=neighbors)
        neigh.fit(tra_X, tra_y)
        score = neigh.score(val_X, val_y)
        print(f'[{algo:^11s} {neighbors:^3d}]\tscore: {score:.4f}')

:::::::: GridSearch
[   auto      3 ]	score: 0.6820
[   auto      5 ]	score: 0.6957
[   auto      7 ]	score: 0.6987
[   auto      9 ]	score: 0.6960
[   auto     11 ]	score: 0.7019
[   auto     13 ]	score: 0.6995
[   auto     15 ]	score: 0.6997
[   auto     17 ]	score: 0.7014
[   auto     19 ]	score: 0.7039
[   auto     21 ]	score: 0.7037
[   auto     23 ]	score: 0.7027
[   auto     25 ]	score: 0.7016
[   auto     27 ]	score: 0.7030
[   auto     29 ]	score: 0.7034
[ ball_tree   3 ]	score: 0.6820
[ ball_tree   5 ]	score: 0.6957
[ ball_tree   7 ]	score: 0.6987
[ ball_tree   9 ]	score: 0.6960
[ ball_tree  11 ]	score: 0.7019
[ ball_tree  13 ]	score: 0.6995
[ ball_tree  15 ]	score: 0.6997
[ ball_tree  17 ]	score: 0.7014
[ ball_tree  19 ]	score: 0.7039
[ ball_tree  21 ]	score: 0.7037
[ ball_tree  23 ]	score: 0.7027
[ ball_tree  25 ]	score: 0.7016
[ ball_tree  27 ]	score: 0.7030
[ ball_tree  29 ]	score: 0.7034
[  kd_tree    3 ]	score: 0.6820
[  kd_tree    5 ]	score: 0.6957
[  kd_tree    7 ]	sc

In [8]:
algo = 'auto'
neighbors = 29
neigh = KNeighborsClassifier(algorithm='auto', n_neighbors=29)
neigh.fit(tra_X, tra_y)
score = neigh.score(val_X, val_y)
print(f'[{algo:^11s} {neighbors:^3d}]\tscore: {score:.4f}')

[   auto     29 ]	score: 0.7101


In [9]:
te1_predict = neigh.predict(te1_scaled)
te2_predict = neigh.predict(te2_scaled)

In [10]:
te1_idx = np.unique(test1.acc_id)
te2_idx = np.unique(test2.acc_id)

In [11]:
df1 = pd.DataFrame({'acc_id':te1_idx, 'isSurvival':te1_predict})
df2 = pd.DataFrame({'acc_id':te2_idx, 'isSurvival':te2_predict})

In [12]:
display(df1.head())
display(df2.head())

,acc_id,isSurvival
0,7,0
1,15,0
2,16,0
3,18,1
4,19,1


,acc_id,isSurvival
0,1,0
1,3,1
2,9,0
3,14,1
4,26,1


In [49]:
tra = train.copy()
te1 = test1.copy()
te2 = test2.copy()
lab = label.copy()

In [16]:
tra = tra.drop('day', axis=1).groupby('acc_id').mean()
te1 = te1.drop('day', axis=1).groupby('acc_id').mean()
te2 = te2.drop('day', axis=1).groupby('acc_id').mean()

In [17]:
tot_sales = lab.amount_spent.sum()
quantile = lab.amount_spent.quantile([.25, .5, .7, .8, .9])
print(quantile)
quantile = quantile.tolist()

print(lab.amount_spent[lab.amount_spent>quantile[-1]].sum() / tot_sales) # 현질 상위 10% 유저의 전체 매출 기여도: 67% --> 상위 10% 이상: heavy user
print(lab.amount_spent[lab.amount_spent>quantile[-2]].sum() / tot_sales) # 현질 상위 20% 유저의 전체 매출 기여도: 83% --> 상위 10%~30% : middle user
print(lab.amount_spent[lab.amount_spent>quantile[-3]].sum() / tot_sales) # 현질 상위 30% 유저의 전체 매출 기여도: 92% --> 상위 30% 이하: non spent user


0.25    0.000000
0.50    0.020343
0.70    0.076985
0.80    0.143745
0.90    0.271187
Name: amount_spent, dtype: float64
0.6764472769057542
0.8368034405895624
0.9229246705260058


In [18]:
def black_cow(x):
    if x > quantile[-1]:
        return 0
    elif x > quantile[-3]:
        return 1
    else:
        return 2

lab['black_cow'] = lab.amount_spent.transform(black_cow)

In [19]:
lab.black_cow.value_counts()

2    28000
1     8000
0     4000
Name: black_cow, dtype: int64

In [20]:
lab = lab['black_cow']

In [21]:
tra_X, val_X, tra_y, val_y = train_test_split(tra_scaled, lab, train_size=0.8)

In [25]:
# gridsearch
param_neighbors=30
param_algorithm= ['auto', 'ball_tree', 'kd_tree', 'brute']

print(':::::::: GridSearch')
for algo in param_algorithm:
    for neighbors in range(3, param_neighbors, 2):
        neigh = KNeighborsClassifier(algorithm=algo, n_neighbors=neighbors)
        neigh.fit(tra_X, tra_y)
        score = neigh.score(val_X, val_y)
        print(f'[{algo:^11s} {neighbors:^3d}]\tscore: {score:.4f}')

:::::::: GridSearch
[   auto      3 ]	score: 0.6871
[   auto      5 ]	score: 0.7076
[   auto      7 ]	score: 0.7180
[   auto      9 ]	score: 0.7218
[   auto     11 ]	score: 0.7282
[   auto     13 ]	score: 0.7329
[   auto     15 ]	score: 0.7350
[   auto     17 ]	score: 0.7395
[   auto     19 ]	score: 0.7399
[   auto     21 ]	score: 0.7416
[   auto     23 ]	score: 0.7422
[   auto     25 ]	score: 0.7424
[   auto     27 ]	score: 0.7430
[   auto     29 ]	score: 0.7444
[ ball_tree   3 ]	score: 0.6871
[ ball_tree   5 ]	score: 0.7076
[ ball_tree   7 ]	score: 0.7180
[ ball_tree   9 ]	score: 0.7218
[ ball_tree  11 ]	score: 0.7282
[ ball_tree  13 ]	score: 0.7329
[ ball_tree  15 ]	score: 0.7350
[ ball_tree  17 ]	score: 0.7395
[ ball_tree  19 ]	score: 0.7399
[ ball_tree  21 ]	score: 0.7416
[ ball_tree  23 ]	score: 0.7422
[ ball_tree  25 ]	score: 0.7424
[ ball_tree  27 ]	score: 0.7430
[ ball_tree  29 ]	score: 0.7444
[  kd_tree    3 ]	score: 0.6871
[  kd_tree    5 ]	score: 0.7076
[  kd_tree    7 ]	sc

In [27]:
algo = 'auto'
neighbors = 29
neigh = KNeighborsClassifier(algorithm='auto', n_neighbors=29)
neigh.fit(tra_X, tra_y)
score = neigh.score(val_X, val_y)
print(f'[{algo:^11s} {neighbors:^3d}]\tscore: {score:.4f}')

te1_predict = neigh.predict(te1_scaled)
te2_predict = neigh.predict(te2_scaled)                        

[   auto     29 ]	score: 0.7444


In [35]:
te1_predict = pd.DataFrame({'acc_id':te1_idx, 'black_cow':te1_predict})
te2_predict = pd.DataFrame({'acc_id':te2_idx, 'black_cow':te2_predict})

In [41]:
df1 = pd.merge(df1, te1_predict, on='acc_id', how='left')
df2 = pd.merge(df2, te2_predict, on='acc_id', how='left')

In [44]:
df1.head()

,acc_id,isSurvival,black_cow
0,7,0,2
1,15,0,2
2,16,0,2
3,18,1,2
4,19,1,2


In [46]:
# 저장
path1 = os.path.join(PREPROCESS_DIR, 'test1-knn-clustering.csv')
path2 = os.path.join(PREPROCESS_DIR, 'test2-knn-clustering.csv')
df1.to_csv(path1, index=False)
df2.to_csv(path2, index=False)